In [1]:
import json
from urllib.request import urlopen
import arcpy
from arcpy import env
import os

In [2]:
arcpy.env.overwriteOutput = True

In [10]:
# function to convert json from url to features
def json2shp(url_string, json_folder, geodatabase, filename):
    response = urlopen(url_string)
    data = response.read()
    txt_str = data.decode('utf-8')
    lines = txt_str.split("\r\n")
    path = json_folder + '/' + filename + '.json'
    fx = open(path, "w")
    for line in lines:
        fx.write(line+ "\n")
    fx.close()
    #arcpy.JSONToFeatures_conversion(path, os.path.join(geodatabase, filename), "Point")

In [4]:
url_string = "https://peacehealthrides.com/opendata/station_information.json"

In [6]:
file_name = "bike_share_station"

In [7]:
geodatabase = r"T:\DCProjects\StoryMap\BikeCounting\BikeMap\ReviewData\ReviewData.gdb"

In [8]:
json_folder = r"T:\DCProjects\StoryMap\BikeCounting\BikeMap"

In [ ]:
# https://pro.arcgis.com/en/pro-app/latest/tool-reference/tool-errors-and-warnings/001001-010000/tool-errors-and-warnings-01551-01575-001558.htm
# not spatial
json2shp(url_string, json_folder, geodatabase, file_name)